In [39]:
import boto3
import s3fs
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

from io import StringIO

role = get_execution_role()

In [40]:
s3 = s3fs.S3FileSystem(anon=False)

bucket = 'ml-presentation/'
key = 'heart_failure_clinical_records_dataset.csv'

df_key = pd.read_csv(s3.open('{}{}'.format(bucket, key)), sep=';')

In [41]:
import sagemaker as sage
from time import gmtime, strftime

from sagemaker.predictor import CSVSerializer

sess = sage.Session()

In [42]:
data_location = 's3://' + bucket + key
data_location

's3://ml-presentation/heart_failure_clinical_records_dataset.csv'

In [43]:
df_key.head()

,heart_failure,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,age
0,1,0,582,0,20,1,265000.00,1.9,130,1,0,4,75.0
1,1,0,7861,0,38,0,263358.03,1.1,136,1,0,6,55.0
2,1,0,146,0,20,0,162000.00,1.3,129,1,1,7,65.0
3,1,1,111,0,20,0,210000.00,1.9,137,1,0,7,50.0
4,1,1,160,1,20,0,327000.00,2.7,116,0,0,8,65.0


In [44]:
df_key.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   heart_failure             299 non-null    int64  
 1   anaemia                   299 non-null    int64  
 2   creatinine_phosphokinase  299 non-null    int64  
 3   diabetes                  299 non-null    int64  
 4   ejection_fraction         299 non-null    int64  
 5   high_blood_pressure       299 non-null    int64  
 6   platelets                 299 non-null    float64
 7   serum_creatinine          299 non-null    float64
 8   serum_sodium              299 non-null    int64  
 9   sex                       299 non-null    int64  
 10  smoking                   299 non-null    int64  
 11  time                      299 non-null    int64  
 12  age                       299 non-null    float64
dtypes: float64(3), int64(10)
memory usage: 30.5 KB


In [45]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']

region = sess.boto_session.region_name

image_uri = '{}.dkr.ecr.{}.amazonaws.com/modelling:latest'.format(account, region)

In [46]:
voting_clf = sage.estimator.Estimator(image_uri,
                       role, 1, 'ml.c4.2xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)

voting_clf.fit(data_location)

2020-11-09 23:23:27 Starting - Starting the training job...
2020-11-09 23:23:28 Starting - Launching requested ML instances......
2020-11-09 23:24:55 Starting - Preparing the instances for training......
2020-11-09 23:25:40 Downloading - Downloading input data
2020-11-09 23:25:40 Training - Downloading the training image..Training complete.

2020-11-09 23:26:13 Uploading - Uploading generated training model
2020-11-09 23:26:13 Completed - Training job completed
Training seconds: 39
Billable seconds: 39


In [47]:
predictor = voting_clf.deploy(1, 'ml.c4.2xlarge', serializer=CSVSerializer())

-----------!

In [49]:
xx=predictor.predict(df_key.iloc[:,1:].values)

In [50]:
xx_str=str(xx,'utf-8')
res_pred = StringIO(xx_str) 

In [51]:
res_pred = pd.read_csv(res_pred, header=None)
res_pred

,0
0,1
1,1
2,1
3,1
4,1
...,...
294,0
295,0
296,0
297,0
